In [1]:
import os

import pandas as pd

import numpy as np

import torch

from torch import tensor

from torch.utils import data

from pandas_plink import read_plink1_bin

import math

In [2]:
# conda install -c conda-forge pandas-plink

In [3]:
%run SNV_Encoding.ipynb

Mapping files: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 68.23it/s]


## Define a class for the tokenised SNV dataset along with some related information

In [4]:
class Tokenised_SNV:
    
    def __init__(self, geno, encoding: int = 2):
    
        if encoding == 4:
        
            token_matrix, diff_mat, non_ref_mat, diff_lens, string_to_tok, tok_to_string, num_toks = get_token_matrix(geno, encoding)
        
            self.diff_alleles_mat = torch.from_numpy(diff_mat)
        
            self.non_ref_alleles_mat = torch.from_numpy(non_ref_mat)
        
            self.diff_lens_mat = torch.from_numpy(diff_lens)
        
        elif encoding != 4:
        
            token_matrix, string_to_tok, tok_to_string, num_toks = get_token_matrix(geno, encoding)
        
        self.token_matrix = token_matrix
    
        self.string_to_tok = string_to_tok
    
        self.tok_to_string = tok_to_string
    
        self.num_toks = num_toks
    
        self.snp_ids = geno.snp.values
    
        self.ids = geno.iid.values  
        

## Loading in the SNV and phenotype datasets

In [5]:
def read_geno_files(encoding: int = 2, test_prop = 0.25, verify_prop = 0.05, subsample_control = False, remove_nan = True):
    
    bed_file = "test_logical.bed"
    
    bim_file = "test_logical.bim"
    
    fam_file = "test_logical.fam"
    
    # Load in genomic files containing SNV data
    
    geno = read_plink1_bin(bed_file, bim_file, fam_file)
    
    # Load in the phenotype information
    
    phenotypes = pd.read_csv("logical_sim_phenos.csv")
    
    # Obtain for ids for each pre-training, training, testing and verifying set
    
    train_ids, test_ids, verify_ids = get_ids_each_set(phenotypes, test_prop, verify_prop)
    
    if subsample_control:
        
        sample_ids = np.concatenate([train_ids, test_ids, verify_ids]).reshape(-1)
        
        train_geno = geno[geno["sample"].isin(sample_ids)]
        
        train_phenos = phenotypes[phenotypes["eid"].isin(sample_ids)]
        
    # Replace NaN with the most common value for alleles
    
    if remove_nan:
        
        flattened_geno = np.ndarray.flatten(train_geno.values)
        
        most_common_value = np.bincount(flattened_geno).argmax()
        
        geno.values[np.isnan(train_geno.values)] = most_common_value
        
    # Obtain the object of SNV token matrix
    
    snv_toks = Tokenised_SNV(geno, encoding)
    
    return snv_toks, phenotypes
    

## Obtain summary statistics for the SNV and phenotype datasets

In [6]:
def summarise_snv_data():
    
    bed_file = "test_logical.bed"
    
    bim_file = "test_logical.bim"
    
    fam_file = "test_logical.fam"
    
    # Load in genomic files containing SNV data
    
    geno = read_plink1_bin(bed_file, bim_file, fam_file)
    
    # Load in the phenotype information
    
    phenotypes = pd.read_csv("logical_sim_phenos.csv")
    
    # The total count
    
    total_num = len(geno.values.reshape(-1))
    
    # Count of the numbers and proportion of each value, including NaN
    
    # 0's
    
    num_zeroes = np.sum(geno.values == 0)
    
    prop_zeroes = 100.0 * num_zeroes / total_num
    
    # 1's
    
    num_ones = np.sum(geno.values == 1)
    
    prop_ones = 100.0 * num_ones / total_num
    
    # 2's
    
    num_twos = np.sum(geno.values == 2)
    
    prop_twos = 100.0 * num_twos / total_num
    
    # NaN's
    
    num_nans = np.sum(np.isnan(geno.values))
    
    prop_nans = num_nans / total_num
    
    print(f"The proportion of 0's in the genomic dataset is {prop_zeroes:.2f} %")
    
    print(f"The proportion of 1's in the genomic dataset is {prop_ones:.2f} %")
    
    print(f"The proportion of 2's in the genomic dataset is {prop_twos:.2f} %")
    
    print(f"The proportion of NaN's in the genomic dataset is {prop_nans:.2f} %")
    
    # Count the number & proportion of having gout
    
    num_gout = np.sum(phenotypes.gout)
    
    prop_gout = num_gout / len(phenotypes)
    
    print(f"The proportion of having gout is {prop_gout:.2f}")

In [7]:
summarise_snv_data()

Mapping files: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 54.56it/s]


The proportion of 0's in the genomic dataset is 33.34 %
The proportion of 1's in the genomic dataset is 33.34 %
The proportion of 2's in the genomic dataset is 33.32 %
The proportion of NaN's in the genomic dataset is 0.00 %
The proportion of having gout is 0.41


In [8]:
bed_file = "test_logical.bed"
    
bim_file = "test_logical.bim"
    
fam_file = "test_logical.fam"
    
# Load in genomic files containing SNV data
    
test_geno = read_plink1_bin(bed_file, bim_file, fam_file)

Mapping files: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 54.58it/s]


In [9]:
test_geno.shape

(10000, 2000)

In [10]:
test_geno.values

array([[1., 1., 2., ..., 0., 2., 2.],
       [2., 2., 0., ..., 1., 1., 0.],
       [1., 1., 1., ..., 2., 1., 0.],
       ...,
       [0., 2., 2., ..., 1., 1., 2.],
       [2., 2., 0., ..., 2., 2., 1.],
       [1., 0., 1., ..., 0., 2., 1.]], dtype=float32)

## Obtain IDs for training, testing and verifying set

## (To ensure equal representation of gout/non-gout proportions in each set)

In [11]:
test_phenons = pd.read_csv("logical_sim_phenos.csv")

In [12]:
test_phenons.head()

,Unnamed: 0,age,sex,bmi,height,gout,eid,urate
0,0,47,Female,20.046436,138.376848,False,0,-1.491421
1,1,43,Male,28.781378,134.027385,True,1,-3.658668
2,2,58,Female,24.268563,183.125706,True,2,-2.546383
3,3,38,Male,31.204586,159.249847,False,3,-2.967442
4,4,44,Female,32.034737,196.405684,False,4,-0.307320


In [13]:
bed_file = "test_logical.bed"
    
bim_file = "test_logical.bim"
    
fam_file = "test_logical.fam"
    
# Load in genomic files containing SNV data
    
fun_geno = read_plink1_bin(bed_file, bim_file, fam_file)
    
# Load in the phenotype information
    
fun_phenons = pd.read_csv("logical_sim_phenos.csv")

Mapping files: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 54.56it/s]


In [14]:
def get_ids_each_set(phenos_set, test_prop, verify_prop):
    
    # Check if csv files of IDs in each set have been created
    
    if os.path.isfile("training_ids.csv") and os.path.isfile("testing_ids.csv") and os.path.isfile("verifying_ids.csv"):
        
        training_ids_df = pd.read_csv("training_ids.csv")
        
        testing_ids_df = pd.read_csv("testing_ids.csv")
        
        verifying_ids_df = pd.read_csv("verifying_ids.csv")
        
    else:
    
        gout = phenos_set[phenos_set["gout"]]
    
        non_gout = phenos_set[~phenos_set["gout"]]
    
        # Random shuffle
    
        gout = gout.sample(frac = 1)
    
        non_gout = non_gout.sample(frac = 1)
    
        # Obtain IDs for training, testing and verifying sets of having gout dataframe
    
        verify_num_gout = math.ceil(verify_prop*len(gout))
    
        verify_test_num_gout = math.ceil((test_prop + verify_prop)*len(gout))
    
        verify_gout, test_gout, train_gout = np.split(gout, [verify_num_gout, verify_test_num_gout])
    
        verify_gout_ids = np.array(verify_gout.eid)
    
        test_gout_ids = np.array(test_gout.eid)
    
        train_gout_ids = np.array(train_gout.eid)
    
        # Obtain IDs for training, testing and verifying sets of non-gout dataframe
    
        verify_num_non_gout = math.ceil(verify_prop*len(non_gout))
    
        verify_test_num_non_gout = math.ceil((test_prop + verify_prop)*len(non_gout))
    
        verify_non_gout, test_non_gout, train_non_gout = np.split(non_gout, [verify_num_non_gout, verify_test_num_non_gout])
    
        verify_non_gout_ids = np.array(verify_non_gout.eid)
    
        test_non_gout_ids = np.array(test_non_gout.eid)
    
        train_non_gout_ids = np.array(train_non_gout.eid)
    
        # Concatenate gout and non-gout sets of IDs
        
        ## Training IDs set
    
        training_ids = np.concatenate((train_gout_ids, train_non_gout_ids))
    
        training_ids.sort()
        
        ## Testing IDs set
    
        testing_ids = np.concatenate((test_gout_ids, test_non_gout_ids))
    
        testing_ids.sort()
        
        ## Verifying IDs set
    
        verifying_ids = np.concatenate((verify_gout_ids, verify_non_gout_ids))

        verifying_ids.sort()
    
        # Convert to Pandas DataFrame and save the 3 sets of IDs to csv file
        
        ## Training IDs set
    
        training_ids_df = pd.DataFrame(training_ids)
    
        training_ids_df.to_csv("./training_ids.csv", header = ["ID"], index = False)
        
        ## Testing IDs set
    
        testing_ids_df = pd.DataFrame(testing_ids)
    
        testing_ids_df.to_csv("./testing_ids.csv", header = ["ID"], index = False)
        
        ## Verifying IDs set
    
        verifying_ids_df = pd.DataFrame(verifying_ids)
    
        verifying_ids_df.to_csv("./verifying_ids.csv", header = ["ID"], index = False)
        
    # Obtain the IDs for training, testing and verifying sets
        
    train_ids = np.array(training_ids_df).reshape(-1)
    
    test_ids = np.array(testing_ids_df).reshape(-1)
    
    verify_ids = np.array(verifying_ids_df).reshape(-1)
    
    return train_ids, test_ids, verify_ids

## Mapping Genome IDs to Positions within the Genome Dataset

In [15]:
def convert_id_to_position(id_array, all_ids):
    
    id_to_position = {}
    
    for pos, val in enumerate(all_ids):
        
        id_to_position[val] = pos
        
    position_list = [id_to_position[str(val)] for val in id_array]
        
    return np.array(position_list)

## Function to convert dataframe of phenotypes into Pytorch tensor object

In [16]:
def convert_pd_to_tensor(phenos_df):
    
    df = phenos_df[["age", "sex", "bmi", "height", "urate"]]
    
    # Encoding sex: Male = 1 ; Female = 2

    df['encoded_sex'] = np.select([df['sex'] == "Male" , df['sex'] == "Female"], [1, 2])
    
    # Dropping the original sex column
    
    df = df.drop('sex', axis=1)
    
    # Converting the dataframe into Pytorch tensor object
    
    phenos_tensor = torch.Tensor(list(df.values))
    
    return phenos_tensor

## Obtain the training, testing & verifying datasets

In [17]:
def get_train_test_verify_data(phenos_set, tokenised_snv_object, test_prop, verify_prop):
    
    # Obtain arrays consisting of IDs in each set
    
    train_ids, test_ids, verify_ids = get_ids_each_set(phenos_set, test_prop, verify_prop)
    
    # Obtain arrays consisting of positions corresponding to IDs in each set
    
    train_pos = convert_id_to_position(train_ids, tokenised_snv_object.ids)
    
    test_pos = convert_id_to_position(test_ids, tokenised_snv_object.ids)
    
    verify_pos = convert_id_to_position(verify_ids, tokenised_snv_object.ids)
    
    # Obtain training, testing & verifying sets of tokenised SNVs + having-gout-or-not
    
    ## Training gout and tokenised SNV set
    
    train_gout = torch.Tensor(phenos_set[phenos_set.eid.isin(train_ids)].gout.values).long()
    
    train_snv = tokenised_snv_object.token_matrix[train_pos, ]
    
    ## Testing gout and tokenised SNV set
    
    test_gout = torch.Tensor(phenos_set[phenos_set.eid.isin(test_ids)].gout.values).long()
    
    test_snv = tokenised_snv_object.token_matrix[test_pos, ]
    
    ## Verifying gout and tokenised SNV set
    
    verify_gout = torch.Tensor(phenos_set[phenos_set.eid.isin(verify_ids)].gout.values).long()
    
    verify_snv = tokenised_snv_object.token_matrix[verify_pos, ]
    
    # Convert training, testing and verifying phenotypes dataframe into Pytorch Tensor Object
    
    ## Training phenotypes set
    
    train_phenos_df = phenos_set[phenos_set.eid.isin(train_ids)]
    
    train_phenos_ts = convert_pd_to_tensor(train_phenos_df)
    
    ## Testing phenotypes set
    
    test_phenos_df = phenos_set[phenos_set.eid.isin(test_ids)]
    
    test_phenos_ts = convert_pd_to_tensor(test_phenos_df)    
    
    ## Verifying phenotypes set
    
    verify_phenos_df = phenos_set[phenos_set.eid.isin(verify_ids)]
    
    verify_phenos_ts = convert_pd_to_tensor(verify_phenos_df)   
    
    ## Obtain the complete datasets of tokenised SNVs, gout and phenotypes
    
    # training_ts = torch.cat((train_snv, train_phenos_ts), 1)
    
    # testing_ts = torch.cat((test_snv, test_phenos_ts), 1)
    
    # verifying_ts = torch.cat((verify_snv, verify_phenos_ts), 1)
    
    return train_snv, train_phenos_ts, train_gout, test_snv, test_phenos_ts, test_gout, verify_snv, verify_phenos_ts, verify_gout


In [18]:
test_phenons.head()

,Unnamed: 0,age,sex,bmi,height,gout,eid,urate
0,0,47,Female,20.046436,138.376848,False,0,-1.491421
1,1,43,Male,28.781378,134.027385,True,1,-3.658668
2,2,58,Female,24.268563,183.125706,True,2,-2.546383
3,3,38,Male,31.204586,159.249847,False,3,-2.967442
4,4,44,Female,32.034737,196.405684,False,4,-0.307320


In [19]:
test_ph = test_phenons[["age", "sex", "bmi", "height", "urate"]]

In [20]:
test_ph.head()

,age,sex,bmi,height,urate
0,47,Female,20.046436,138.376848,-1.491421
1,43,Male,28.781378,134.027385,-3.658668
2,58,Female,24.268563,183.125706,-2.546383
3,38,Male,31.204586,159.249847,-2.967442
4,44,Female,32.034737,196.405684,-0.307320


In [21]:
snv_object = Tokenised_SNV(test_geno, encoding = 5)

In [22]:
snv_object.ids

array(['0', '1', '2', ..., '9997', '9998', '9999'], dtype=object)

In [23]:
tr_snv,tr_phenos,tr_gout,te_snv,te_phenos,te_gout,ve_snv,ve_phenos,ve_gout = get_train_test_verify_data(fun_phenons, snv_object, test_prop = 0.01, verify_prop = 0.00)

C:\Users\tuanh\AppData\Local\Temp\ipykernel_24560\2141829978.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['encoded_sex'] = np.select([df['sex'] == "Male" , df['sex'] == "Female"], [1, 2])
C:\Users\tuanh\AppData\Local\Temp\ipykernel_24560\2141829978.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)
  phenos_tensor = torch.Tensor(list(df.values))
C:\Users\tuanh\AppData\Local\Temp\ipykernel_24560\2141829978.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [23]:
np.unique(test_geno.pos.values)

array([0])

In [24]:
test_geno.a0.values

array(['A', 'G', 'C', ..., 'T', 'T', 'A'], dtype=object)

In [25]:
test_geno.a1.values

array(['C', 'A', 'C', ..., 'G', 'T', 'A'], dtype=object)

In [26]:
test_geno.values

array([[1., 1., 2., ..., 0., 2., 2.],
       [2., 2., 0., ..., 1., 1., 0.],
       [1., 1., 1., ..., 2., 1., 0.],
       ...,
       [0., 2., 2., ..., 1., 1., 2.],
       [2., 2., 0., ..., 2., 2., 1.],
       [1., 0., 1., ..., 0., 2., 1.]], dtype=float32)

In [29]:
train_ids, test_ids, verify_ids = get_ids_each_set(fun_phenons, test_prop = 0.25, verify_prop = 0.05)

In [30]:
train_ids

array([   1,    3,    5, ..., 9994, 9995, 9996], dtype=int64)

In [31]:
test_ids.shape

(2500,)

In [32]:
verify_ids.shape

(501,)

In [44]:
tr_snv[0:5]

tensor([[ 5,  3,  5,  ..., 17, 18,  3],
        [ 5,  3,  5,  ..., 17,  6, 10],
        [ 5,  4,  9,  ...,  4, 18, 10],
        [ 5,  4,  9,  ...,  6,  6,  3],
        [ 3,  8,  5,  ..., 17, 18,  3]], dtype=torch.int32)

In [47]:
te_snv[0:5]

tensor([[ 7,  8,  5,  ...,  6,  6,  3],
        [ 7,  8,  9,  ...,  4, 18,  3],
        [ 3,  3,  5,  ...,  6,  6,  3],
        [ 3,  3,  5,  ..., 17,  6,  3],
        [ 3,  3,  5,  ..., 17,  6,  3]], dtype=torch.int32)

In [49]:
import torch.nn as nn

import torch.nn.functional as F

In [50]:
class One_Hot_Encoding(nn.Module):
    
    def __init__(self, num_classes: int):
        
        super().__init__()
        
        self.num_classes = num_classes
        
    def forward(self, input_ts):
        
        return F.one_hot(input_ts.long(), num_classes = self.num_classes)

In [51]:
pros = One_Hot_Encoding(num_classes = 23)

In [61]:
ax = pros(tr_snv)

In [66]:
torch.set_printoptions(threshold=10)

In [71]:
tr_snv.shape

torch.Size([6999, 2000])

In [72]:
ax.shape

torch.Size([6999, 2000, 23])

In [75]:
tr_snv[0]

tensor([ 5,  3,  5,  ..., 17, 18,  3], dtype=torch.int32)

In [86]:
torch.set_printoptions(threshold=1000)

ax[0][-2]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [100]:
test_phenons

,age,sex,bmi,height,gout,urate
0,47,Female,20.046436,138.376848,False,-1.491421
1,43,Male,28.781378,134.027385,True,-3.658668
2,58,Female,24.268563,183.125706,True,-2.546383
3,38,Male,31.204586,159.249847,False,-2.967442
4,44,Female,32.034737,196.405684,False,-0.307320
...,...,...,...,...,...,...
9995,38,Female,21.260590,113.790969,False,-1.176638
9996,47,Male,32.446960,168.947474,False,-0.503730
9997,42,Female,28.356535,175.415825,True,-3.719973
9998,58,Male,24.583989,184.753297,False,-0.145249


In [101]:
test_phenons['sex'].replace(['Female', 'Male'], [0, 1], inplace=True)

C:\Users\tuanh\AppData\Local\Temp\ipykernel_5096\1805010914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_phenons['sex'].replace(['Female', 'Male'], [0, 1], inplace=True)


In [102]:
test_phenons = test_phenons[["age", "sex", "bmi", "height", "gout", "urate"]]

In [103]:
test_phenons

,age,sex,bmi,height,gout,urate
0,47,0,20.046436,138.376848,False,-1.491421
1,43,1,28.781378,134.027385,True,-3.658668
2,58,0,24.268563,183.125706,True,-2.546383
3,38,1,31.204586,159.249847,False,-2.967442
4,44,0,32.034737,196.405684,False,-0.307320
...,...,...,...,...,...,...
9995,38,0,21.260590,113.790969,False,-1.176638
9996,47,1,32.446960,168.947474,False,-0.503730
9997,42,0,28.356535,175.415825,True,-3.719973
9998,58,1,24.583989,184.753297,False,-0.145249


In [104]:
data_df = test_phenons[["age", "sex", "bmi", "height", "urate"]]

data_df

,age,sex,bmi,height,urate
0,47,0,20.046436,138.376848,-1.491421
1,43,1,28.781378,134.027385,-3.658668
2,58,0,24.268563,183.125706,-2.546383
3,38,1,31.204586,159.249847,-2.967442
4,44,0,32.034737,196.405684,-0.307320
...,...,...,...,...,...
9995,38,0,21.260590,113.790969,-1.176638
9996,47,1,32.446960,168.947474,-0.503730
9997,42,0,28.356535,175.415825,-3.719973
9998,58,1,24.583989,184.753297,-0.145249


In [105]:
lab_df = test_phenons[["gout"]]

lab_df

,gout
0,False
1,True
2,True
3,False
4,False
...,...
9995,False
9996,False
9997,True
9998,False


In [119]:
tr_snv

tensor([[ 5,  3,  5,  ..., 17, 18,  3],
        [ 5,  3,  5,  ..., 17,  6, 10],
        [ 5,  4,  9,  ...,  4, 18, 10],
        ...,
        [ 3,  8,  5,  ...,  6,  6,  3],
        [ 3,  8,  9,  ...,  6, 18,  3],
        [ 3,  4,  5,  ..., 17, 18,  3]], dtype=torch.int32)

In [123]:
np_tr_snv = tr_snv.numpy()

np_tr_gout = tr_gout.numpy()

np_te_snv = te_snv.numpy()

np_te_gout = te_gout.numpy()

In [111]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

In [125]:
X_train, X_test, y_train, y_test = train_test_split(np_tr_snv, np_tr_gout, test_size = .2)

In [126]:
model = LogisticRegression().fit(X_train, y_train) 

C:\Users\tuanh\anaconda3\envs\pytorch\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [127]:
model.intercept_

array([-0.00065923])

In [128]:
model.coef_

array([[-0.01757824, -0.03809569,  0.0025418 , ...,  0.01748201,
         0.00501555, -0.03898065]])

In [129]:
y_pred = model.predict(X_test) 

y_pred 

array([1, 0, 0, ..., 1, 1, 0], dtype=int64)

In [130]:
np.unique(y_pred)

array([0, 1], dtype=int64)

In [132]:
score = model.score(X_test, y_test)
print(score)

0.5721428571428572


In [133]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(y_test, y_pred)

matrix.diagonal()/matrix.sum(axis=1)

array([0.62280702, 0.50498339])

In [134]:
from sklearn.metrics import classification_report

In [138]:
print(classification_report(y_test, y_pred, target_names = ['False', 'True'], digits=4))

              precision    recall  f1-score   support

       False     0.6252    0.6228    0.6240       798
        True     0.5025    0.5050    0.5037       602

    accuracy                         0.5721      1400
   macro avg     0.5638    0.5639    0.5639      1400
weighted avg     0.5724    0.5721    0.5723      1400

